In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import numpy as np
import warnings 

C:\Users\Hemshree\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('C:\\Users\\Hemshree\\Data Science\\data\\hindireduced.csv')
images_data = data.iloc[:,:-1].values
train_labels = data.iloc[:,-1].values
labels = LabelBinarizer().fit_transform(train_labels)
#labels = OneHotEncoder().fit_transform(labels).todense()


In [3]:
train_images, test_images, train_labels, test_labels = train_test_split(images_data, labels, test_size = 0.25, random_state = 0)
sess = tf.InteractiveSession()
x = tf.placeholder("float", shape=[None, 1024])
y_ = tf.placeholder("float", shape=[None, 6])


In [33]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.007)
    return tf.Variable(initial)
    
def bias_variable(shape):
    initial = tf.constant(0.00018, shape = shape)
    return tf.Variable(initial)
    
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

In [38]:
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

#Variable and bias 
W_conv1 = weight_variable([5, 5, 1, 32]) #5*5 matrix for 32 features and depth 1
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,32,32,1])
#First convolution
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([5, 5, 32, 64])#5*5 matrix for 64 features and depth 32
b_conv2 = bias_variable([64])
#second convolution
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

#densely connected layer, where we allow all neurons to merge and process entire image
h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 6])
b_fc2 = bias_variable([6])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(100):
    batch0, batch1 = next_batch(6)
    train_step.run(feed_dict={x: batch0, y_: batch1, keep_prob: 0.5})

print ("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_images, y_: test_labels, keep_prob: 1.0}))


test accuracy 0.9375
